In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig,eigh

from spatial_filtering import arrays, constants, direction_of_arrival
import importlib

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)


# Constants
f = 1e9              # Frequency 1 GHz
wavelength = constants.c / f

# ULA parameters
N = 400             # Number of elements
d = 10 * wavelength / 2   # Spacing

array = arrays.UniformLinearArray(N, d)

# Near-field source parameters
r_source = [200, 20, 4000]        # meters
theta_source_deg = [20, 75, 40]
theta_source = np.deg2rad(theta_source_deg)

r_steps = 5001
theta_steps = 720

# Signal parameters
SNR_dB = 40
SNR = 10**(SNR_dB / 10)
snapshots = 2000


X = np.zeros((N, snapshots), dtype=np.complex128)
# Generate signal
for (r, theta) in zip(r_source, theta_source):
    a = array.nf_steering_vector(r, [0, theta], wavelength).reshape(-1,1)
    signal = (np.random.randn(1, snapshots) + 1j * np.random.randn(1, snapshots)) / np.sqrt(2)  # complex Gaussian signal
    X += a @ signal
noise_power = 1 / SNR
noise = np.sqrt(noise_power / 2) * (np.random.randn(N, snapshots) + 1j * np.random.randn(N, snapshots))

X += noise

# Received signal
# Covariance matrix
R = X @ X.conj().T / N #np.cov(X)

output = direction_of_arrival.MUSICNF2D().get_direction(
    array,
    R, 
    num_interferers=3,
    wavelength = wavelength,
    r_min=0,
    r_max = 5000,
    r_steps=r_steps,
    theta_min_deg = 0,
    theta_max_deg = 90,
    theta_steps=theta_steps,
    phi_min_deg = 0,
    phi_max_deg = 0,
    phi_steps=1,
)


# Plot
output.sort(by="Q", descending = True).head(20)

In [ ]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  # or 'iframe' or 'colab' depending on your environment

# Filter top 1% Q values (adjust as needed)
threshold = output['Q'].quantile(0.99)
high_Q = output.filter(pl.col('Q') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='r',      # range
    z='Q',      # MUSIC power
    color='Q',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Range (meters)',
        zaxis_title='Q (dB)'
    )
)

fig.show()